<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mistral_integration_with_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://neon.tech/docs/extensions/pg_embedding#migrate-from-pg_embedding-to-pgvector


In [1]:
#Install Libraries to access Google Drive and OpenAI resources.
%pip install colab-env --upgrade --quiet --root-user-action=ignore
import colab_env

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
def insert_document(id,embedding):
    #review_embedding=get_embedding(text)
    ### INSERT INTO DB
    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    conn = ps.connect(database=DB_NAME,
							user=DB_USER,
							password=DB_PASS,
							host=DB_HOST,
							port=DB_PORT)
    cur = conn.cursor() # creating a cursor
    cur.execute("""
        INSERT INTO documents
        (id, embedding)
        VALUES ('%s',
                '%s')""" % (id,embedding))

    conn.commit()
    print("INSERT EMBEDDING %s successfully"%embedding)
    conn.close()
    cur.close()

In [3]:
def query_selection(query):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""
          %s """%query)
      records = cur.fetchall()
      print("Total rows are:  ", len(records))
      print("Printing each row")
      print()
      n=0
      for row in records:
          n=n+1
          print("ROW %s: "%n, row)
      conn.close()
      cur.close()
      print()
      print("QUERY SELECTION successfully")
      print()

In [4]:
def drop_extension(extension):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""DROP EXTENSION IF EXISTS %s CASCADE"""%extension)
      cur.query
      conn.commit()
      cur.close()
      conn.close()

      print()
      print("DROP EXTENSION %s successfully"%extension)
      print()

In [5]:
# install PSQL WITH DEV Libraries AND PG embedding
!apt install postgresql postgresql-contrib &>log
!service postgresql restart
!sudo apt install postgresql-server-dev-all

 * Restarting PostgreSQL 14 database server
   ...done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
postgresql-server-dev-all is already the newest version (238).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [6]:
!cp -pr /content/gdrive/MyDrive/tools/pg_embedding /content/
%cd /content/pg_embedding/
print()
print('START: PG embedding COMPILATION')
!make
!make install # may need sudo
#!make uninstall
print('END: PG embedding COMPILATION')
print()

/content/pg_embedding

START: PG embedding COMPILATION
make: Nothing to be done for 'all'.
/bin/mkdir -p '/usr/lib/postgresql/14/lib'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/usr/bin/install -c -m 755  embedding.so '/usr/lib/postgresql/14/lib/embedding.so'
/usr/bin/install -c -m 644 .//embedding.control '/usr/share/postgresql/14/extension/'
/usr/bin/install -c -m 644 .//embedding--0.3.5--0.3.6.sql .//embedding--0.3.5.sql .//embedding--0.3.6.sql  '/usr/share/postgresql/14/extension/'
/bin/mkdir -p '/usr/lib/postgresql/14/lib/bitcode/embedding'
/bin/mkdir -p '/usr/lib/postgresql/14/lib/bitcode'/embedding/
/usr/bin/install -c -m 644 embedding.bc '/usr/lib/postgresql/14/lib/bitcode'/embedding/./
/usr/bin/install -c -m 644 hnswalg.bc '/usr/lib/postgresql/14/lib/bitcode'/embedding/./
/usr/bin/install -c -m 644 distfunc.bc '/usr/lib/postgresql/14/lib/bitcode'/embedding/./
cd '/usr/lib/postgresql/14/lib/bitcode' && /usr/lib/llvm-14/

In [7]:
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")

/content
ALTER ROLE

Extensions Available:
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('se

In [8]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#!sudo -u postgres psql -c "DROP EXTENSION vector CASCADE"
#!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding real[])"

h="{1,2,3}"
hh= "INSERT INTO documents(id, embedding) VALUES (1,'%s'), (2,'{4,5,6}')"%h
print(hh)

#del insert_document
insert_document(1,'{1,2,3}')
insert_document(2,'{4,5,6}')

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5)"

/content
ALTER ROLE
ERROR:  extension "embedding" already exists
DROP TABLE
CREATE TABLE
INSERT INTO documents(id, embedding) VALUES (1,'{1,2,3}'), (2,'{4,5,6}')
INSERT EMBEDDING {1,2,3} successfully
INSERT EMBEDDING {4,5,6} successfully


In [9]:
# install PSQL WITH DEV Libraries AND PG embedding
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

!cp -pr /content/gdrive/MyDrive/tools/pgvector /content/
%cd /content/pgvector/
print()
print('START: PG VECTOR COMPILATION')
!make
!make install
#!make uninstall
print('END: PG VECTOR COMPILATION')
print()

/content/pgvector

START: PG VECTOR COMPILATION
make: Nothing to be done for 'all'.
/bin/mkdir -p '/usr/lib/postgresql/14/lib'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/usr/bin/install -c -m 755  vector.so '/usr/lib/postgresql/14/lib/vector.so'
/usr/bin/install -c -m 644 .//vector.control '/usr/share/postgresql/14/extension/'
/usr/bin/install -c -m 644 .//sql/vector--0.1.0--0.1.1.sql .//sql/vector--0.1.1--0.1.3.sql .//sql/vector--0.1.3--0.1.4.sql .//sql/vector--0.1.4--0.1.5.sql .//sql/vector--0.1.5--0.1.6.sql .//sql/vector--0.1.6--0.1.7.sql .//sql/vector--0.1.7--0.1.8.sql .//sql/vector--0.1.8--0.2.0.sql .//sql/vector--0.2.0--0.2.1.sql .//sql/vector--0.2.1--0.2.2.sql .//sql/vector--0.2.2--0.2.3.sql .//sql/vector--0.2.3--0.2.4.sql .//sql/vector--0.2.4--0.2.5.sql .//sql/vector--0.2.5--0.2.6.sql .//sql/vector--0.2.6--0.2.7.sql .//sql/vector--0.2.7--0.3.0.sql .//sql/vector--0.3.0--0.3.1.sql .//sql/vector--0.3.1--0.3.2.sql .//sql/v

In [10]:
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"

print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")


Extensions Available:
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('ssli

In [11]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"
!sudo -u postgres psql -c "CREATE EXTENSION vector"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding vector(3))"

insert_document(1,'[1,2,3]')
insert_document(2,'[4,5,6]')
!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw (embedding vector_l2_ops)"

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (m=3)"


/content
ALTER ROLE
DROP EXTENSION
CREATE EXTENSION
DROP TABLE
CREATE TABLE
INSERT EMBEDDING [1,2,3] successfully
INSERT EMBEDDING [4,5,6] successfully
CREATE INDEX


In [12]:
## <-> and <=> Euclidean (L2) and Cosine distances NO Manhattan distance: <~>
query_selection("SELECT * FROM documents ORDER BY embedding::vector <-> '[3,1,2]' LIMIT 5")

Total rows are:   2
Printing each row

ROW 1:  (1, '[1,2,3]')
ROW 2:  (2, '[4,5,6]')

QUERY SELECTION successfully



In [13]:
print('')
print("Extensions Available:")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print('')
print("Extensions Used:")
query_selection("SELECT * FROM pg_extension")


Extensions Available:
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 39:  ('seg',)
ROW 40:  ('ssli

In [14]:
# Euclidean (L2) distance index:
#CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <-> array[3,3,3] LIMIT 1;

# Cosine distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <=> array[3,3,3] LIMIT 1;

# Manhattan distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_manhattan_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <~> array[3,3,3] LIMIT 1;



# AWS

In [15]:
#!rm -rf /content/*.pdf
!mkdir -p /content/data/
%cd /content/data/

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "/content/data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    #print(file_path)
    urlretrieve(url, file_path)

/content/data


In [16]:
%pip install openai==0.28  --root-user-action=ignore
%pip install langchain
%pip install "unstructured[all-docs]"
%pip install tiktoken
%pip install -q -U sentence-transformers

In [17]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

In [18]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

#%cd /content/data

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs)}')

# of Document Pages 25
# of Document Chunks: 299


In [19]:
#%pip install langchain
#%pip install colab-env
#%pip install openai==0.28  --root-user-action=ignore

# install PSQL WITH DEV Libraries AND PG embedding
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

import openai

import os
collection_name='AWS'

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

%pip install colab-env
import colab_env

connection_string = os.getenv("DATABASE_URL")

# https://supabase.com/blog/fewer-dimensions-are-better-pgvector
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

%cd /content/

# install PSQL WITH DEV Libraries AND PG embedding
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#DROP EXTENSION IF EXISTS
#drop_extension('embedding')
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"
!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS vector CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding;"
#!sudo -u postgres psql -c "CREATE EXTENSION vector;"

print()
#del embeddings
print("Extension available")
query_selection("SELECT name FROM pg_available_extensions order by 1")
print()
print("Extension used")
query_selection("SELECT * FROM pg_extension order by 1")
print()

/content
ALTER ROLE


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


NOTICE:  drop cascades to column embedding of table documents
DROP EXTENSION
CREATE EXTENSION

Extension available
Total rows are:   48
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('

In [20]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)

query = "How has AWS evolved?"
print()
print("-" * 80)
print(query)

#query = db.embedding_function.embed_query(query)
results_with_scores = db.similarity_search_with_score(query)

for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

print()
print("-" * 80)


--------------------------------------------------------------------------------
How has AWS evolved?
Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5201687


Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5201687


Content: in AWS. Our new customer pipeline is robust, as are our active migrations. Many companies usediscontinuous periods like this to step back and determine what they strategically want to change, and wefind an increasing number of enterprises opting out of managing their own infrastructure

In [ ]:
#https://platform.openai.com/docs/guides/text-generation

!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet
!pip install pypdf

%pip install openai==0.28  --root-user-action=ignore
%pip install tiktoken
%pip install colab-env --upgrade --quiet --root-user-action=ignore

In [22]:
#%pip install colab-env --upgrade --quiet --root-user-action=ignore
#!pip install accelerate

import torch
from textwrap import fill
from IPython.display import Markdown, display

import colab_env
import os

access_token = os.getenv("HF_TOKEN")

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

#from transformers import AutoTokenizer, MistralForCausalLM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME='mistralai/Mistral-7B-Instruct-v0.1'
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 512
generation_config.temperature = 0.9
generation_config.top_p = 0.9
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

#model.to(device)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    pad_token_id=tokenizer.eos_token_id
)

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipeline)

In [25]:
query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query1 = "who is the President of the USA?"
query2 = "Who won the baseball World Series in 2020? and Who Lost"

device="cuda"
def prompt_completion(query):
    messages = [
        {"role": "user", "content": "%s"%query}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    #https://stackoverflow.com/questions/69609401/suppress-huggingface-logging-warning-setting-pad-token-id-to-eos-token-id

    generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                    pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.batch_decode(generated_ids)
    print()
    print()
    result=decoded[0].replace('<s> [INST] %s [/INST]'%query,"")
    result=result.replace('</s>',"")
    print('Prompt: %s'%query)
    print('-'*80)
    print('Answer: %s'%result)

prompt_completion(query)
print()
print('='*80)
prompt_completion(query1)
print()
print('='*80)
prompt_completion(query2)

query3 = "what is the 20.5% of 40?"
query4 = "As a data scientist, can you explain the concept of regularization in machine learning?"
query5 ='Which country has the most natural lakes? Answer with only the country name.'

print()
print('='*80)
prompt_completion(query3)
print()
print('='*80)
prompt_completion(query4)
print()
print('='*80)
prompt_completion(query5)


query6 = "How AWS has evolved?"
print()
print('='*80)
prompt_completion(query6)




Prompt: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------
Answer:  Let's go over this step-by-step:

1. We know that each cone costs $1.25.
2. Six cones cost 6 * $1.25 = $7.50.
3. The question states that you paid with a $10 bill.
4. When you pay with a $10 bill for something that costs $7.50, you get back $2.50 in change (because $10 - $7.50 = $2.50).

The final answer is that you got back $2.50.



Prompt: who is the President of the USA?
--------------------------------------------------------------------------------
Answer:  As of my most recent update, the President of the United States is Joe Biden. However, I don't have the capability to access real-time information, so please double-check this information if you need it for current purposes.



Prompt: Who won the baseball World Series in 2020? and Wh

In [26]:
import colab_env
import os

access_token = os.getenv("HF_TOKEN")
device = "cuda" # the device to load the model onto

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig, GenerationConfig, pipeline
from transformers import AutoTokenizer, MistralForCausalLM

#model = MistralForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",padding_side="left")
#tokenizer.pad_token = tokenizer.eos_token # Most LLMs don't have a pad token by default


prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# https://github.com/huggingface/transformers/blob/main/src/transformers/models/mistral/modeling_mistral.py

# Generate
#print()
#generate_ids = model.generate(inputs.input_ids, max_length=30)

#generate_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
#                pad_token_id=tokenizer.eos_token_id)

#response=tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#print(response)
print()

#query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
query="What is your favourite condiment?"
messages = [
    {"role": "user", "content": "%s"%query},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
#model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=512, do_sample=True, negative_prompt_attention_mask='attention_mask',
                pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(decoded[0])


[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!  [INST] Do you have mayonnaise recipes? [/INST] Yes, I have a mayonnaise recipe that's simple to make and always turns out delicious! Here it is:

Ingredients:

* 1 cup of mayonnaise
* 1 tablespoon of yellow mustard
* 2 tablespoons of apple cider vinegar
* 1 small clove of garlic (optional)
* 1/2 teaspoon of salt (optional)
* 1/4 teaspoon of black pepper (optional)

Instructions:

1. In a mixing bowl, whisk together the mayonnaise, yellow mustard, apple cider vinegar, and garlic (if using) until well combined.
2. If you prefer a smoother consistency, you can use an immersion blender or a regular blender to blend the mixture until it's smooth.
3. If you prefer a chunkier consistency, simply skip step 2.
4. Taste the mixture and adjust the seasoning with salt and black pepper to taste.
5.

In [33]:
#!pip install transformers --upgrade
#device = "cuda"
from IPython.display import Markdown, display

query = "who is Barack Obama?"
result = llm(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))
print()
print("-" * 80)
print()



<b>who is Barack Obama?</b>

<p>
Answer:
Barack Obama was the 44th President of the United States. He served from 2009 to 2017 and was the first African American to hold the office. Prior to his presidency, he served as a senator for Illinois.</p>


--------------------------------------------------------------------------------



In [36]:
from langchain.llms import OpenAI
#import colab_env
from IPython.display import Markdown, display

from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

#retriever = db.as_retriever(search_type="similarity_score_threshold",search_kwargs='1')
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create a chain to answer questions
#qa = RetrievalQA.from_chain_type(
#     llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

qa = RetrievalQA.from_chain_type(
     llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "How AWS has evolved?"

result = qa(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

print()
print('chain to answer questions')
print("-" * 80)
result = qa({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

<b>How AWS has evolved?</b>

<p>{'query': 'How AWS has evolved?', 'result': ' AWS has evolved from simply offering customers more functionality in AWS than they could find elsewhere to being a much more game-changing offering than it was previously. This evolution has been driven by customer needs and has allowed AWS to continue to differentiate itself from competitors.', 'source_documents': [Document(page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.', metadata={'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}), Document(page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.', metadata={'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'})]}</p>


chain to answer questions
--------------------------------------------------------------------------------
Query: How AWS has evolved?

Result:  AWS has evolved from a cloud computing platform with some unique features into a comprehensive suite of tools and services for building, deploying, managing and scaling applications across multiple domains such as cloud computing, edge computing, machine learning, artificial intelligence, serverless, data analytics, data storage, databases, and many others. This evolution has made it possible for customers to leverage the power of AWS technology for their specific needs and requirements, thus providing them with a competitive advantage over other companies in their industry.

Context Documents: 
page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.' metadata={'year': 2021, 'source': 

In [29]:
#query='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'

#User
#I bought an ice cream for six kids. Each cone was $1.25, and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

#Run instructions
#You are a personal math tutor. Write and run code to answer math questions.

#Math Tutor
#First, we need to calculate the total cost for six ice cream cones, each priced at $1.25. We'll multiply the price per cone by the number of cones purchased: [ $1.25 \times 6 \text{ cones} = \text{Total cost} ]

#Having calculated the total cost, we will then subtract this amount from the $10 bill to find out how much change should be returned: [ $10 - \text{Total cost} = \text{Change returned} ]

#Now, let's do the calculation.

#code_interpreter(
# Given values price_per_cone = 1.25 number_of_cones = 6 amount_paid = 10 # Total cost of the cones total_cost = price_per_cone * number_of_cones # Change returned change_returned = amount_paid - total_cost change_returned
#) --. 2.5

#Math Tutor
#After calculating, the total change you should get back after purchasing the six ice cream cones is $2.50.

#User
#Write a Python code for that solution.

#Math Tutor
#Certainly! Here's the Python code that demonstrates the solution we've just discussed:

# Given values
price_per_cone = 1.25
number_of_cones = 6
amount_paid = 10

# Total cost for the ice cream cones
total_cost = price_per_cone * number_of_cones

# Change to be returned after payment
change_returned = amount_paid - total_cost

# Output the result
print(f"The change returned after buying the ice cream is: ${change_returned}")

The change returned after buying the ice cream is: $2.5
